# 통합 및 자동화된 파이썬 코드
이 코드는 처음에 원본 파일 경로와 최종 저장 경로만 입력하면 모든 과정이 중단 없이 진행됩니다.

In [1]:
import re
import os
import sys
import subprocess
import importlib
import requests
import urllib3

# --- 0. 환경 설정 및 라이브러리 로드 ---
try:
    from deep_translator import GoogleTranslator
except ImportError:
    print("필수 라이브러리(deep-translator) 설치 중...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "deep-translator", "requests", "--user"])
    importlib.invalidate_caches()
    from deep_translator import GoogleTranslator

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def force_no_ssl_verify():
    old_request = requests.Session.request
    def new_request(self, method, url, *args, **kwargs):
        kwargs['verify'] = False
        return old_request(self, method, url, *args, **kwargs)
    requests.Session.request = new_request

force_no_ssl_verify()

# --- I. 숫자 시퀀스 괄호 처리 로직 ---
def enclose_numbers_in_parentheses(text):
    pattern = r"(\b(?!claims?\b)[a-zA-Z]+\b)(\s\d+[a-z0-9\-\']*(?:[\s,]+(?:and|or|to|-)\s+\d+[a-z0-9\-\']*(?!\w)|[\s,]+\d+[a-z0-9\-\']*(?!\w))*)"
    
    def replacement_logic(match):
        prefix = match.group(1)
        number_sequence = match.group(2).strip() 
        return f"{prefix}({number_sequence})"

    return re.sub(pattern, replacement_logic, text, flags=re.IGNORECASE)

# --- II. 영한 번역 로직 ---
def chunk_text(text, max_size=5000):
    chunks = []
    start = 0
    while start < len(text):
        lookahead_end = min(start + max_size, len(text))
        segment = text[start:lookahead_end]
        if lookahead_end == len(text):
            if segment.strip(): chunks.append(segment.strip())
            break
        best_split_index = -1
        for char in ['.', '?', '!']:
            last_punc_index = segment.rfind(char)
            if last_punc_index > best_split_index:
                best_split_index = last_punc_index
        if best_split_index != -1:
            best_end = start + best_split_index + 1
            chunks.append(text[start:best_end].strip())
            start = best_end
        else:
            chunks.append(segment.strip())
            start = lookahead_end
    return chunks

def translate_text(text):
    translator = GoogleTranslator(source='en', target='ko')
    chunks = chunk_text(text)
    translated_chunks = []
    for i, chunk in enumerate(chunks):
        print(f"   > 번역 진행 중... ({i+1}/{len(chunks)})")
        try:
            translated_chunks.append(translator.translate(chunk))
        except Exception as e:
            translated_chunks.append(f"[번역 오류: {e}]")
    return "\n\n".join(translated_chunks)

# --- III. 비격식체 변환 로직 ---
def transform_korean_sentence(text):
    conversion_rules = {
        "입니다": "이다", "합니다": "한다", "있습니다": "있다",
        "됩니다": "된다", "냅니다": "낸다", "시킵니다": "시킨다",
        "필요한다": "필요하다", "유리한다": "유리하다",
        "했습니다": "했다" , "줍니다": "준다",
        "없습니다": "없다", "않습니다": "않는다",
        "무화과.": "도", "무화과": "도", "그림": "도",
        "도. 도": "도",
        "매개변수": "파라미터",  "종방향": "길이방향",
        "방사상으로": "반경방향으로", "집니다": "진다"
    }
    for formal, casual in conversion_rules.items():
        text = text.replace(formal, casual)
    return text

# --- 메인 실행 프로세스 ---
def main():
    print("="*60)
    print(" 특허/기술 문서 통합 번역 프로세서 (괄호처리 -> 번역 -> 문체변환)")
    print("="*60)

    # 1. 경로 설정
    input_path = input("✅ 원본 영문 파일 경로를 입력하세요: ").strip()
    if not os.path.exists(input_path):
        print("파일이 존재하지 않습니다.")
        return

    # 중간 파일 및 최종 파일 경로 자동 설정 (사용자 편의)
    base_name = os.path.splitext(input_path)[0]
    step1_path = f"{base_name}_step1_en_bracketed.txt"
    step2_path = f"{base_name}_step2_ko_formal.txt"
    final_path = input(f"✅ 최종 결과 저장 경로를 입력하세요 (엔터 시 '{base_name}_final_ko_casual.txt'로 저장): ").strip()
    if not final_path:
        final_path = f"{base_name}_final_ko_casual.txt"

    # --- STEP 1: 괄호 처리 ---
    print("\n[STEP 1] 숫자 시퀀스 괄호 처리 중...")
    with open(input_path, 'r', encoding='utf-8') as f:
        en_text = f.read()
    
    bracketed_text = enclose_numbers_in_parentheses(en_text)
    with open(step1_path, 'w', encoding='utf-8') as f:
        f.write(bracketed_text)
    print(f" 완료: {step1_path}")

    # --- STEP 2: 번역 (STEP 1의 출력을 입력으로 사용) ---
    print("\n[STEP 2] 영한 번역 진행 중 (Google)...")
    translated_text = translate_text(bracketed_text)
    with open(step2_path, 'w', encoding='utf-8') as f:
        f.write(translated_text)
    print(f" 완료: {step2_path}")

    # --- STEP 3: 문체 변환 (STEP 2의 출력을 입력으로 사용) ---
    print("\n[STEP 3] 비격식체 종결형 변환 중...")
    final_text = transform_korean_sentence(translated_text)
    with open(final_path, 'w', encoding='utf-8') as f:
        f.write(final_text)
    
    print("\n" + "="*60)
    print(f"모든 작업이 완료되었습니다!")
    print(f"최종 저장 위치: {final_path}")
    print("="*60)

if __name__ == "__main__":
    main()
    

 특허/기술 문서 통합 번역 프로세서 (괄호처리 -> 번역 -> 문체변환)


✅ 원본 영문 파일 경로를 입력하세요:  D:\업무\영문명세서\2026 영문명세서\IP-2026-0014-DE-000.txt
✅ 최종 결과 저장 경로를 입력하세요 (엔터 시 'D:\업무\영문명세서\2026 영문명세서\IP-2026-0014-DE-000_final_ko_casual.txt'로 저장):  



[STEP 1] 숫자 시퀀스 괄호 처리 중...


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa1 in position 923: invalid start byte

In [1]:
import re
import os
import sys
import subprocess
import importlib
import requests
import urllib3

# --- 0. 환경 설정 및 라이브러리 로드 ---
try:
    from deep_translator import GoogleTranslator
except ImportError:
    print("필수 라이브러리(deep-translator) 설치 중...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "deep-translator", "requests", "--user"])
    importlib.invalidate_caches()
    from deep_translator import GoogleTranslator

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def force_no_ssl_verify():
    old_request = requests.Session.request
    def new_request(self, method, url, *args, **kwargs):
        kwargs['verify'] = False
        return old_request(self, method, url, *args, **kwargs)
    requests.Session.request = new_request

force_no_ssl_verify()

# --- I. 숫자 시퀀스 괄호 처리 로직 ---
def enclose_numbers_in_parentheses(text):
    pattern = r"(\b(?!claims?\b)[a-zA-Z]+\b)(\s\d+[a-z0-9\-\']*(?:[\s,]+(?:and|or|to|-)\s+\d+[a-z0-9\-\']*(?!\w)|[\s,]+\d+[a-z0-9\-\']*(?!\w))*)"
    
    def replacement_logic(match):
        prefix = match.group(1)
        number_sequence = match.group(2).strip() 
        return f"{prefix}({number_sequence})"

    return re.sub(pattern, replacement_logic, text, flags=re.IGNORECASE)

# --- II. 영한 번역 로직 ---
def chunk_text(text, max_size=5000):
    chunks = []
    start = 0
    while start < len(text):
        lookahead_end = min(start + max_size, len(text))
        segment = text[start:lookahead_end]
        if lookahead_end == len(text):
            if segment.strip(): chunks.append(segment.strip())
            break
        best_split_index = -1
        for char in ['.', '?', '!']:
            last_punc_index = segment.rfind(char)
            if last_punc_index > best_split_index:
                best_split_index = last_punc_index
        if best_split_index != -1:
            best_end = start + best_split_index + 1
            chunks.append(text[start:best_end].strip())
            start = best_end
        else:
            chunks.append(segment.strip())
            start = lookahead_end
    return chunks

def translate_text(text):
    translator = GoogleTranslator(source='en', target='ko')
    chunks = chunk_text(text)
    translated_chunks = []
    for i, chunk in enumerate(chunks):
        print(f"   > 번역 진행 중... ({i+1}/{len(chunks)})")
        try:
            translated_chunks.append(translator.translate(chunk))
        except Exception as e:
            translated_chunks.append(f"[번역 오류: {e}]")
    return "\n\n".join(translated_chunks)

# --- III. 비격식체 변환 로직 ---
def transform_korean_sentence(text):
    conversion_rules = {
        "입니다": "이다", "합니다": "한다", "있습니다": "있다",
        "됩니다": "된다", "냅니다": "낸다", "시킵니다": "시킨다",
        "필요한다": "필요하다", "유리한다": "유리하다",
        "했습니다": "했다" , "줍니다": "준다",
        "없습니다": "없다", "않습니다": "않는다",
        "무화과.": "도", "무화과": "도", "그림": "도",
        "도. 도": "도",
        "매개변수": "파라미터",  "종방향": "길이방향",
        "방사상으로": "반경방향으로", "집니다": "진다"
    }
    for formal, casual in conversion_rules.items():
        text = text.replace(formal, casual)
    return text

# --- 메인 실행 프로세스 ---
def main():
    print("="*60)
    print(" 특허/기술 문서 통합 번역 프로세서 (괄호처리 -> 번역 -> 문체변환)")
    print("="*60)

    # 1. 경로 설정
    input_path = input("✅ 원본 영문 파일 경로를 입력하세요: ").strip()
    if not os.path.exists(input_path):
        print("파일이 존재하지 않습니다.")
        return

    # 중간 파일 및 최종 파일 경로 자동 설정 (사용자 편의)
    base_name = os.path.splitext(input_path)[0]
    step1_path = f"{base_name}_step1_en_bracketed.txt"
    step2_path = f"{base_name}_step2_ko_formal.txt"
    final_path = input(f"✅ 최종 결과 저장 경로를 입력하세요 (엔터 시 '{base_name}_final_ko_casual.txt'로 저장): ").strip()
    if not final_path:
        final_path = f"{base_name}_final_ko_casual.txt"

    # --- STEP 1: 괄호 처리 ---
    print("\n[STEP 1] 숫자 시퀀스 괄호 처리 중...")
    with open(input_path, 'r', encoding='utf-8') as f:
        en_text = f.read()
    
    bracketed_text = enclose_numbers_in_parentheses(en_text)
    with open(step1_path, 'w', encoding='utf-8') as f:
        f.write(bracketed_text)
    print(f" 완료: {step1_path}")

    # --- STEP 2: 번역 (STEP 1의 출력을 입력으로 사용) ---
    print("\n[STEP 2] 영한 번역 진행 중 (Google)...")
    translated_text = translate_text(bracketed_text)
    with open(step2_path, 'w', encoding='utf-8') as f:
        f.write(translated_text)
    print(f" 완료: {step2_path}")

    # --- STEP 3: 문체 변환 (STEP 2의 출력을 입력으로 사용) ---
    print("\n[STEP 3] 비격식체 종결형 변환 중...")
    final_text = transform_korean_sentence(translated_text)
    with open(final_path, 'w', encoding='utf-8') as f:
        f.write(final_text)
    
    print("\n" + "="*60)
    print(f"모든 작업이 완료되었습니다!")
    print(f"최종 저장 위치: {final_path}")
    print("="*60)

if __name__ == "__main__":
    main()
    

 특허/기술 문서 통합 번역 프로세서 (괄호처리 -> 번역 -> 문체변환)


✅ 원본 영문 파일 경로를 입력하세요:  D:\업무\영문명세서\2026 영문명세서\IP-2026-0014-DE-000.txt
✅ 최종 결과 저장 경로를 입력하세요 (엔터 시 'D:\업무\영문명세서\2026 영문명세서\IP-2026-0014-DE-000_final_ko_casual.txt'로 저장):  



[STEP 1] 숫자 시퀀스 괄호 처리 중...


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa1 in position 923: invalid start byte